In [2]:
from torch.nn import functional as F
from collections import OrderedDict
import torch.nn as nn
import torch
import os

In [3]:
git_home = os.getcwd() # get current directory 
file = f'{git_home}/bigram_model.pt'

In [4]:
## This code is was given as part of the architecture.py ##

vocab = list("0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!#$%&'()*+,-./:;<=>?@[\]^_`{|}\n")

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        logits = self.token_embedding_table(idx) 
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits

**Looking at the _architecture.py_ code that was given, you see that the function returns `logits`.** 

**In machine learning, `logits` are model outputs (predictions) that are not normalized. Usually, there would be additional steps that will interpret these `logits` into a valid predition (usually a number between 0 and 1).**   

In [5]:
# load model
model = BigramLanguageModel(len(vocab)) # run model
state_dict = torch.load(file) # load state_dict from the .pt file 
model.load_state_dict(state_dict) # load the given state_dict into the model 

<All keys matched successfully>

------------
**In the block below, we will use the seed value `1337`, which was given in the _architecture.py_ script. Adding a manual seed instead of a random seed allows for reproduciblity (will get the same answer every time). Whenever there is random seed, the answers are different.**

--------
**The following lines:** <br>
    `logits = model(idx)` <br>
    `probs = F.softmax(logits.squeeze(), dim=0)` <br>

**This corresponds to what was said above about logits, there needs to be an additional step for prediction purposes. This brings us to `SoftMax`, which is a very common function used in classification algorithms.** 

**`SoftMax` takes an input Tensor and rescales it so that the elements of the n-dimensional output Tensor lie in the range [0,1] and sum to 1.**

_For more info on SoftMax, check out https://pytorch.org/docs/stable/generated/torch.nn.Softmax.html ._

------
**The following line:** <br>
`index = torch.multinomial(probs, num_samples=1).item()` <br>
**This line indexes a multinomial probability to each sample from the `probs` variable. This index will then be used for the list of characters that was given in the _archticeture.py_ script.**

``vocab = list("0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!#$%&'()*+,-./:;<=>?@[\]^_`{|}\n")``

**Each character will have a probability associated with it and will be caluclated based on a character set with `{`**
**The reason I added the `generated_text = ["{"]`, is because we _know_ the flag will have `{` in it. Because of this, I wanted to help it out by finding the highest probability of characters that _only_ include a `{` in it.**

_For more on multinomial, check out https://pytorch.org/docs/stable/generated/torch.multinomial.html ._

In [6]:
# set manual seed for reproducibility
torch.manual_seed(1337) # this seed value was given in the architecture.py file

# generate text with the manual seed and the state_dict given to the model

generated_text = ["{"] # added this because we know that the flag will have `{`. 
idx = torch.tensor([[vocab.index(generated_text[-1])]])
for i in range(50):
    logits = model(idx)
    probs = F.softmax(logits.squeeze(), dim=0)
    index = torch.multinomial(probs, num_samples=1).item()
    token = vocab[index]
    generated_text.append(token)
    idx = torch.tensor([[index]])
    
# join tokens to form the generated text
generated_text = "".join(generated_text)
print(generated_text)

{Pr0t3c7_L1fe}
HTB{Pr0t3c7_L1fe}
HTB{Pr0t3c7_L1fe}



**૮₍˶ •. • ⑅₎ა ♡ glockachu**